In [6]:
import re

In [7]:
with open('structure_tree.txt', 'r') as f:
    lines = f.readlines()
    lines = [l.rstrip() for l in lines if len(l) > 0]
    
    current = {}
    structure_list = {}
    
    for l in lines:
        if l.startswith('          - '):
            level = 3
            text = l.strip()[2:]
        elif l.startswith('     - '):
            level = 2
            text = l.strip()[2:]
        elif l.startswith('     '):
            level = 1
            text = l.strip()
        else:
            level = 0
            text = l.strip()
            
        if len(text) == 0:
            continue
        
        try:
            res = re.findall('^(.+?)(:\s?(.*))?$', text)[0]
        except:
            print 'text ERROR:', text
            raise
            
        name, _, desc = res
        
        if '[BEST]' in desc:
            goodness = 'best'
        elif '[GOOD]' in desc:
            goodness = 'good'
        else:
            goodness = None
            
        
        current[level] = name
        
        try:
            fullname, abbreviation = re.findall('^(.*),\s?(.*?)$', name)[0]
        except:
            fullname = name
            abbreviation = ''
        
        parent = current[level-1] if level > 0 else None
        
        structure_list[name] = {'level': level, 
               'fullname': fullname, 
               'abbr': abbreviation, 
               'description': desc,
                'goodness': goodness,
              'parent': parent,
                'children': []}
        
        if parent is not None:
            structure_list[parent]['children'].append(name)
        

In [8]:
structure_list['brainstem'] = {'fullname': 'brainstem', 
                               'level':-1, 
                               'children': [name for name, s in structure_list.iteritems() if s['level'] == 0]}

In [9]:
import json
json.dump(structure_list, open('structure_tree.json', 'w'))

In [10]:
print 'Best structures:'

for name, s in structure_list.iteritems():
    if s['goodness'] == 'best':
        print s['abbr']
        
print '\n'
print 'Good structures:'
        
for name, s in structure_list.iteritems():
    if s['goodness'] == 'good':
        print s['abbr']

Best structures:
fr
Amb
DC
Tz
3N
Pn
RtTg
SNR
7n
AP
10N
VLL
4N


KeyError: 'goodness'

In [12]:
def print_dict(node):
    print '\t'*node['level'] + node['fullname']
    if len(node['children']) == 0:
        return
    else:
        for name in node['children']:
            print_dict(structure_list[name])
    
for name, s in structure_list.iteritems():
    if s['level'] == 0:
        print_dict(s)

Medulla
	vestibular nuclei
		superior
		lateral
		medial
		spinal
		interstitial nucleus of the vestibular nerve
		nucleus y
	dorsal motor nucleus of vagus
	Central grey of Medulla
	hypoglossal nucleus
	perihypoglossal complex
		prepositus nucleus
		Intercalated Nucleus
		nucleus of Roller/sublingual nucleus
	diffuse reticular nucleus/nucleus of Kolliker-Fuse
	paratrigeminal nucleus
	petritrigeminal nucleus
	inferior salivatory nucleus
	ambiguus nucleus
	Gracile nucleus
	cuneate nucleus
	external cuneate nucleus
	Spinal trigeminal nucleus, caudal part
	Spinal trigeminal nucleus, interpolar part
	area postrema
	central canal
	medullary white matters
	inferior olivary complex
		principle inferior olive nucleus
		dorsal inferior olive nucleus
		medial inferior olive nucleus
	Raphe nuclei of the medulla
		raphe magnus
		raphe obscurus
		raphe pallidus
	Solitary nucleus
		dorsolateral
		dorsomedial
		medial
		parvicellular
		ventrolateral
	Parasolitary nucleus
	Cochlear nuclei
		dorsal coch

In [13]:
def extract_names(d, node, structure_tree_dict):

    if len(node['children']) == 0:
        return
    else:
        for name in node['children']:
            print name
            
            if 'abbr' in structure_tree_dict[name] and len(structure_tree_dict[name]['abbr']) > 0:
                key = structure_tree_dict[name]['fullname'] + ' (' + structure_tree_dict[name]['abbr'] + ')'
            else:
                key = structure_tree_dict[name]['fullname']
            
            if key not in d:
                d[key] = {}
                
            extract_names(d[key], structure_tree_dict[name], structure_tree_dict)

In [14]:
structure_tree_dict = json.load(open('structure_tree.json'))
structure_tree_names = {'brainstem': {}}

extract_names(structure_tree_names['brainstem'], structure_tree_dict['brainstem'], structure_tree_dict)

Medulla
vestibular nuclei, Ve
superior, SVe
lateral, LVe
medial, MVe
spinal, SpVe
interstitial nucleus of the vestibular nerve, I8
nucleus y, Y
dorsal motor nucleus of vagus, 10N
Central grey of Medulla, CG
hypoglossal nucleus, 12N
perihypoglossal complex, PhC
prepositus nucleus, Pr
Intercalated Nucleus, In
nucleus of Roller/sublingual nucleus, Ro
diffuse reticular nucleus/nucleus of Kolliker-Fuse, KF
paratrigeminal nucleus, Pa5
petritrigeminal nucleus
inferior salivatory nucleus, IS
ambiguus nucleus, Amb
Gracile nucleus, Gr
cuneate nucleus, Cu
external cuneate nucleus, ECu
Spinal trigeminal nucleus, caudal part, Sp5C
Spinal trigeminal nucleus, interpolar part, Sp5I
area postrema, AP
central canal, CC
medullary white matters
inferior olivary complex, IO
principle inferior olive nucleus, IOPr
dorsal inferior olive nucleus, IOD
medial inferior olive nucleus, IOM
Raphe nuclei of the medulla
raphe magnus, RMg
raphe obscurus, ROb
raphe pallidus, RPa
Solitary nucleus, Sol
dorsolateral
dorsom

In [15]:
structure_tree_names

{'brainstem': {u'Basal Pons': {u'Pontine Nuclei (Pn)': {},
   u'longitudinal pontine fibers': {},
   u'middle cerebellar peduncle (mcp)': {},
   u'transverse pontine fibers (tfp)': {}},
  u'Medulla': {u'Central grey of Medulla (CG)': {},
   u'Cochlear nuclei': {u'dorsal cochlear nucleus (DC)': {},
    u'ventral cochlear nucleus (VC)': {u'ventral cochlear nucleus, anterior (VCA)': {},
     u'ventral cochlear nucleus, posterior (VCP)': {}}},
   u'Gracile nucleus (Gr)': {},
   u'Parasolitary nucleus (PSol)': {},
   u'Raphe nuclei of the medulla': {u'raphe magnus (RMg)': {},
    u'raphe obscurus (ROb)': {},
    u'raphe pallidus (RPa)': {}},
   u'Solitary nucleus (Sol)': {u'dorsolateral': {},
    u'dorsomedial': {},
    u'medial': {},
    u'parvicellular': {},
    u'ventrolateral': {}},
   u'Spinal trigeminal nucleus, caudal part (Sp5C)': {},
   u'Spinal trigeminal nucleus, interpolar part (Sp5I)': {},
   u'ambiguus nucleus (Amb)': {},
   u'area postrema (AP)': {},
   u'central canal (CC)':

In [ ]:
def get_children_names(name):
        
    node = structure_tree_dict[name]
    if 'abbr' in node and len(node['abbr']) > 0:
        key = node['fullname'] + ' (' + node['abbr'] + ')'
    else:
        key = node['fullname']
    return (key, dict([get_children_names(child_name) for child_name in node['children']]))

In [ ]:
structure_tree_names = dict([get_children_names('brainstem')])